In [ ]:
from crewai import Agent, Task, Crew
import requests
import os

# Replace 'YOUR_NEWSAPI_KEY' with your actual NewsAPI key
os.environ['OPENAI_API_KEY'] = 'api_key'

NEWS_API_KEY = '3api_key'

# Function to validate URLs before adding them to the summary
def validate_url(url):
    try:
        response = requests.head(url, allow_redirects=True)
        if response.status_code == 200:
            return True
    except requests.RequestException:
        return False
    return False

# Function to fetch tech news using NewsAPI
def fetch_tech_news():
    url = f"https://newsapi.org/v2/everything?q=AI+technology&from=2024-10-18&to=2024-10-18&apiKey={NEWS_API_KEY}"
    response = requests.get(url)
    
    if response.status_code == 200:
        data = response.json()
        articles = data.get('articles', [])
        
        news_summary = []
        for article in articles:
            title = article.get('title', 'No title available')
            description = article.get('description', 'No description available')
            link = article.get('url', 'No link available')
            source = article.get('source', {}).get('name', 'Unknown source')
            published_at = article.get('publishedAt', 'Unknown date')
            
            # Check if the URL is valid before adding it to the summary
            if validate_url(link):
                news_summary.append(f"### **{title}**\n")
                news_summary.append(f"**Published by:** {source} | **Date:** {published_at}\n")
                news_summary.append(f"{description}\n")
                news_summary.append(f"[Read more here]({link})\n\n")
            else:
                news_summary.append(f"### **{title}**\n")
                news_summary.append(f"**Published by:** {source} | **Date:** {published_at}\n")
                news_summary.append(f"{description}\n")
                news_summary.append(f"**Link unavailable**\n\n")
        
        if news_summary:
            # Construct a more detailed blog post
            blog_post = (
                "# The Scoop on AI: What Went Down on October 18, 2024\n\n"
                "Hello, tech enthusiasts! Artificial Intelligence (AI) is no longer just a concept from science fiction; "
                "it's an integral part of our daily lives. Whether it's Netflix suggesting your next binge-watch or Siri assisting "
                "you with reminders, AI is shaping our interactions in countless ways. In this blog post, we will explore the "
                "most significant AI updates and breakthroughs that occurred on October 18, 2024.\n\n"
                
                "## The Day's Big Deal: A Groundbreaking AI Update\n"
                "On this day, Google unveiled an innovative AI model that promises to revolutionize weather forecasting. This state-of-the-art model, powered by advanced neural networks, "
                "is designed to analyze an extensive array of datasets, including satellite imagery and historical weather data. The implications of this development are immense. By providing "
                "more accurate and timely weather predictions, communities can better prepare for extreme weather events, thereby enhancing disaster management strategies. For instance, "
                "a local government can utilize this model to issue warnings and allocate resources more effectively, potentially saving lives and minimizing economic impacts. The integration "
                "of such technology highlights the transformative power of AI in addressing real-world challenges.\n\n"
            )
            blog_post += "\n".join(news_summary)
            blog_post += (
                "As we reflect on the rapid advancements in AI, it's clear that we are entering an era of smarter technologies. These innovations are not only designed to perform tasks but also "
                "to analyze data and make predictions that were previously unimaginable. The horizon is filled with opportunities as we begin to harness AI's full potential across various sectors.\n\n"
                
                "### Conclusion\n"
                "In summary, the developments in AI this month exemplify its significant impact across diverse industries. As technology continues to evolve, it becomes increasingly crucial for "
                "businesses and individuals to stay informed and adapt to these changes. The future promises to be exciting, with AI poised to redefine how we live and work. So, stay tuned for more updates as we "
                "witness the ongoing revolution in the AI landscape.\n\n"
                
                "### References\n"
                "- Official press releases from Google, IBM, and OpenAI\n"
                "- Tech news outlets like TechCrunch, Wired, and The Verge\n"
                "- Research papers and articles in AI-related scientific journals\n"
                "- AI-focused blogs and forums for the hottest trends and developments\n"
                "Don't miss out on the advancements shaping our world! Stay curious and engaged as the landscape of AI continues to evolve."
            )
            return blog_post
        else:
            return "No relevant news found."
    else:
        return f"Error fetching data. Status code: {response.status_code}"

class TechNewsAgent(Agent):
    def __init__(self):
        super().__init__(
            role="Tech News Fetcher",
            goal="Fetch the latest news and updates about AI and technology.",
            backstory="You are responsible for fetching the latest news and updates about AI and other technologies. "
                      "Provide a comprehensive summary of trends and ongoing developments with relevant links.",
            allow_delegation=False,
            verbose=True
        )

    def fetch_data(self):
        return fetch_tech_news()

# Create the new tech news agent
tech_news_agent = TechNewsAgent()

# Existing agents
planner = Agent(
    role="Content Planner",
    goal="Plan engaging and factually accurate content on {topic}.",
    backstory="You're working on planning a blog article about the topic: {topic}. "
              "Your work is the basis for the Content Writer to write an article on this topic.",
    allow_delegation=False,
    verbose=True
)

writer = Agent(
    role="Content Writer",
    goal="Write insightful and factually accurate opinion piece about the topic: {topic}.",
    backstory="You're working on a new opinion piece about the topic: {topic}. "
              "You base your writing on the work of the Content Planner.",
    allow_delegation=False,
    verbose=True
)

editor = Agent(
    role="Editor",
    goal="Edit a given blog post to align with the writing style of the organization.",
    backstory="You are an editor who reviews a blog post from the Content Writer.",
    allow_delegation=False,
    verbose=True
)

# Define the tasks
plan = Task(
    description=(
        "1. Prioritize the latest trends, key players, and noteworthy news on {topic}.\n"
        "2. Identify the target audience, considering their interests and pain points.\n"
        "3. Develop a detailed content outline including an introduction, key points, and a call to action.\n"
        "4. A well-written blog post in markdown format, ready for publication, each section should have 1 to 2 big paragraphs with 6 lines of words, each paragraph should have 6 to 8 lines.\n"
        "5. Providing relevant references at the end is mandatory.\n"
        "6. Include SEO keywords and relevant data or sources."
    ),
    expected_output="A comprehensive content plan document with an outline, audience analysis, SEO keywords, and resources.",
    agent=planner,
)

write = Task(
    description=(
        "1. Use the content plan to craft a compelling blog post on {topic}.\n"
        "2. Incorporate SEO keywords naturally.\n"
        "3. Sections/Subtitles are properly named in an engaging manner.\n"
        "4. Ensure the post is structured with an engaging introduction, insightful body, and a summarizing conclusion.\n"
        "5. Proofread for grammatical errors and alignment with the brand's voice.\n"
        "6. Providing relevant references at the end is mandatory."
    ),
    expected_output="A well-written blog post in markdown format, ready for publication, each section should have 1 to 2 big paragraphs with 6 lines of words, each paragraph should have 6 to 8 lines.",
    agent=writer,
)

edit = Task(
    description=("Proofread the given blog post for grammatical errors and alignment with the brand's voice."),
    expected_output="A well-written blog post in markdown format, ready for publication, each section should have 1 to 2 paragraphs with 6 lines of words, each paragraph should have 6 to 8 lines.",
    agent=editor
)

# Creating the crew, including the new TechNewsAgent
crew = Crew(
    agents=[planner, writer, editor, tech_news_agent],
    tasks=[plan, write, edit],
    verbose=2
)

# Fetching the latest tech news
latest_tech_news = tech_news_agent.fetch_data()
print("Latest Tech News:\n", latest_tech_news)

# Start the crew process with a topic
result = crew.kickoff(inputs={"topic": "Give me some trending topic names for the year 2024 regarding AI and Technology with thier links so that i can write a blog on them. Also provide me relevant references at last."})

from IPython.display import Markdown
Markdown(result)


2024-10-18 18:56:48,690 - 8496 - __init__.py-__init__:538 - WARNING: Overriding of current TracerProvider is not allowed


Latest Tech News:
 No relevant news found.
 [DEBUG]: == Working Agent: Content Planner
 [INFO]: == Starting Task: 1. Prioritize the latest trends, key players, and noteworthy news on Give me some trending topic names for the year 2024 regarding AI and Technology with thier links so that i can write a blog on them. Also provide me relevant references at last..
2. Identify the target audience, considering their interests and pain points.
3. Develop a detailed content outline including an introduction, key points, and a call to action.
4. A well-written blog post in markdown format, ready for publication, each section should have 1 to 2 big paragraphs with 6 lines of words, each paragraph should have 6 to 8 lines.
5. Providing relevant references at the end is mandatory.
6. Include SEO keywords and relevant data or sources.


> Entering new CrewAgentExecutor chain...
To provide the best content plan, I will first conduct a thorough research on the latest trends in AI and technology for th

# The Future is Now: AI Trends Shaping 2024

Artificial Intelligence (AI) is no longer a futuristic concept. Presently, it's shaping our daily lives and industries in unforeseen ways. As we step into 2024, the dependence on and integration of AI in various domains are becoming more pronounced. Let's explore the trends that are making headlines this year.

## The Emergence of Autonomous Things in 2024

The concept of autonomous things is no longer confined to the realm of sci-fi movies as we embrace 2024. Drones, self-driving cars, and smart appliances have become commonplace, transforming our lifestyle and work processes. While these autonomous things are enhancing efficiency, they also pose new challenges concerning safety, ethics, and legislation. The year 2024 marks a significant milestone in our journey towards a more automated world. It's intriguing to watch how this trend unfolds ([McKinsey Global Institute, 2017](https://www.mckinsey.com/~/media/McKinsey/Industries/Advanced%20Electronics/Our%20Insights/How%20artificial%20intelligence%20can%20deliver%20real%20value%20to%20companies/MGI-Artificial-Intelligence-Discussion-paper.ashx)).

## AI's Impact on Healthcare: Transforming the Medical Field in 2024

The influence of AI on healthcare is profound, offering promising breakthroughs in disease diagnosis, drug discovery, and personalized medicine. As we step into 2024, AI's role in healthcare has become even more pivotal, helping to save millions of lives and improve the quality of care. However, concerns about data privacy and the ethical use of AI in healthcare continue to be heated debate topics ([The Lancet Digital Health, 2019](https://www.thelancet.com/journals/landig/article/PIIS2589-7500(19)30084-5/fulltext)).

## AI's Role in Education: The Future of Learning in 2024

AI is revolutionizing the educational landscape, making learning more engaging, personalized, and accessible. In 2024, AI's potential in education reaches new heights with advancements in virtual tutors, AI-powered educational games, and data-driven classroom insights. However, we must ensure that these AI-driven changes are equitable and do not widen the existing digital divide ([UNESCO, 2019](https://unesdoc.unesco.org/ark:/48223/pf0000366994)).

## AI's Influence on Business: How AI is Driving the Future of Work in 2024

AI has become an essential driver for business innovation, efficiency, and competitiveness in 2024. From automating routine tasks to generating business insights, AI is changing the way businesses operate. However, the rapid adoption of AI raises concerns about job displacement and the need for reskilling ([IET, 2018](https://www.theiet.org/impact-society/sectors/information-communications/artificial-intelligence/)).

As we navigate through 2024, these trends in AI are not only shaping our immediate future but also prompting important questions about how we should adapt to this AI-driven world. Stay updated on these trends and consider the implications and opportunities AI presents across various fields. The future of AI is here, and it's time we embrace it fully.